# Purpose:

> This notebook can be used to start up the connection between the S1-testbed instruction generation model and the S1-DT model demo using the MQTT protocol and the RECOIL server. 

# Imports

In [1]:
'''
    This simulates a client that consumes the `pub` of the MQTT Pub Server.

    Sample output:
        Data received:  0.003
        Data received:  0.004
        Data received:  0.003
'''
import numpy as np
from transformers import pipeline
from LLM_TOOLS.LLM_Bots import *
import paho.mqtt.client as mqtt
import time
import os

2025-01-22 20:45:00.382329: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 20:45:00.418202: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-22 20:45:01.033831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:

random.randint(1, 100)

35

# Start connection and send/recieve loop with LLM

## NOTE:
> The 'S1_MotorMaintenaceInstructor' folder with the corresponding model is assumed to be locally stored in the same directory as this file. 

In [2]:
#### Create LLM instructor bot

# get current directory 
current_dir = os.getcwd()

# set path to LLM 
print("The current path is: ", current_dir)
model_path = current_dir +  "/"
Model_Name = model_path + r"S1_MotorMaintenaceInstructor"

# get instance of instructor bot while passing either the path to the 
# relevant folder, or the path to a model on hugging face which should consist of either the base name, or a username/model_name
# for instance for me this would be gjonesQ02/S1_InsturctionGeneratorGamma
instructor_Bot = InstructorBot(bot_path=Model_Name, name='LLM-veridian',)


# callback upon connection
def on_connect(client, userdata, flags, rcode):
    """
        Method used to connect to and subscribe to the needed topics on the MQTT connection
    """
    print("Connected flags ",str(flags),"result code ",str(rcode))
    if (rcode != 0): # handle error
        print("MQTT auth failed")
        os._exit(1) # works - hard exit
    else:
        print("Connection successful!")
        service_str = topic
        client.subscribe(service_str) 
        print(f"Subscribed to: {service_str}")
        return



# callback upon message
def on_message(client, userdata, message):
    """
        Method used to process the message recieved on the broker
        and determine which instruction set to send. The method 
        will only query the LLM if a new fault is seen. 
        
    """
    print(f"Message received: {message.payload.decode('utf-8')}")
    try:
        status = int(message.payload.decode("utf-8"))
        status_decoder = {
            0: "baseline",
            1: "bent_shaft",
            2: "eccentric_rotor",
            3: "offset_misalignment",
            4: "resonance_beam",
            5: "imbalance",
            6: "faulted_coupling",
            7: "faulted_bearing",
            8: "angular_misalignment",
            9: "looseness",
        }

        # pull string version of status from int version mapping
        status = status_decoder[status]
        response = userdata["response"]
        
        # check for new motor state
        # if so actual get a generation
        # otherwise use the old one
        if status != userdata['previous']:
            # old_status = status
            userdata['previous'] = status
            print("new status received: ", status)
            print("Please wait for the response...")
            response, prompt = instructor_Bot.respond_to_task_query(status, rng=False, selection=1, ret_prompt=True)
            
            if "\n\n" in response:
                print("double found")
                responseL = response.split("\n\n")
                response = responseL[0] + responseL[1]
            if "\n\n\n" in response:
                print("triple found")
                responseL = response.split("\n\n\n")
                response = responseL[0] + response[1]
            print("Response:\n", response)
            userdata["response"] = response
            time.sleep(5)
        else:
            print("No new status received....")
            # print("Previous status: ", userdata['previous'])
            # print("Current status: ", status)
            print("Response:\n", response)
            time.sleep(5)
        # client.publish("instructions", response.encode('utf-8'))
        print(f"Publishing to {to_publish}")
        client.publish(to_publish, response.encode('utf-8'), qos=2)
    except Exception as ex:
        print(f"ex: {ex}")
    

mqttBroker = MQTT_BROKER

client = mqtt.Client(client_id="LLM_Client5yuyuyuy") # sample client, unique name
client.username_pw_set(username=MQTT_USER,password=MQTT_PWD)
client.user_data_set({"name": "LLM_Client4", "previous":"", "response":''}) # sample user data
client.on_message=on_message 
client.on_connect = on_connect
# client.subscribe("motor_state")
# client.subscribe("prediction")

client.connect(mqttBroker, MQTT_PORT, MQTT_KEEPALIVE) 


# client.loop_forever()
client.loop_start()

# # Let's grab some data: e.g., status, ch2_velocityPk
# # Note that we may/should sub to each topic in a sub client.
# #


# # This demonstrates how to get multiple (topic, qos) in an array
# # client.subscribe([("status", 1),("ch2_velocityPk", 1)]) 


# delay the send/recivev loop after 3600 seconds (6 mins)
# time.sleep(3600) # only run for 3600 secs (as a demo; remove for PROD
time.sleep(7200) # only run for 3600 secs (as a demo; remove for PROD
client.loop_stop()

The current path is:  /home/gerald/ML_ACT


/tmp/ipykernel_2624828/3240339138.py:121: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id="LLM_Client5yuyuyuy") # sample client, unique name


Connected flags  {'session present': 0} result code  0
Connection successful!
Subscribed to: prediction
Message received: 2
new status received:  eccentric_rotor
Please wait for the response...
double found
Response:
 Job: eccentric_rotor
1) Remove the top section of both bearing pedestals taking care to save the 0.003 shims installed in the bearing housing.
2) Then loosen the coupling set screw on the rotor shaft side of the coupling.
3) Lift the rotor shaft assembly of the MVAT assembly and pull the shaft out of the coupling.
4) Then loosen the outer bearing and rotor set screws and remove them from the rotor shaft.
5) Install the new rotor in place of the eccentric rotor and re-install the outer bearing.
6) Re-install the rotor assembly.
7) Be sure to install the 0.003 shims.
Publishing to instructions
Message received: 2
No new status received....
Response:
 Job: eccentric_rotor
1) Remove the top section of both bearing pedestals taking care to save the 0.003 shims installed in the

KeyboardInterrupt: 